In [3]:
%matplotlib inline
import os
import h5py
import numpy as np
#import dataset_processing.data_freiburg_subject_ordering as subjects_ordering
#import dataset_processing.config.system as sys_config
from scipy.ndimage import gaussian_filter


import SimpleITK as sitk
import math
from scipy import interpolate

from matplotlib import pyplot as plt
from skimage.morphology import skeletonize_3d, binary_dilation, dilation, cube

from utils import normalize_image


from utils_centerline import (load_masked_data, show_center_lines,load_cropped_data_sliced, 
load_cropped_data_straightened, extract_slice_from_sitk_image, load_masked_data_sliced,
create_center_lines, show_center_lines)

In [4]:
control_seg_path = "/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady/segmenter_rw_pw_hard/controls"
control_path = "/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady/preprocessed/controls/numpy"
basepath = "/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady"
print(os.listdir(control_seg_path)), print(os.listdir(control_path))

['seg_MACDAVD_101_.npy', 'seg_MACDAVD_102_.npy', 'seg_MACDAVD_103_.npy', 'seg_MACDAVD_104_.npy', 'seg_MACDAVD_105_.npy', 'seg_MACDAVD_106_.npy']
['MACDAVD_101_.npy', 'MACDAVD_102_.npy', 'MACDAVD_103_.npy', 'MACDAVD_104_.npy', 'MACDAVD_105_.npy', 'MACDAVD_106_.npy', 'MACDAVD_107_.npy', 'MACDAVD_108_.npy', 'MACDAVD_109_.npy', 'MACDAVD_145_.npy', 'MACDAVD_148_.npy', 'MACDAVD_150_.npy', 'MACDAVD_152_.npy', 'MACDAVD_154_.npy', 'MACDAVD_110_.npy', 'MACDAVD_111_.npy', 'MACDAVD_113_.npy', 'MACDAVD_114_.npy', 'MACDAVD_115_.npy', 'MACDAVD_116_.npy', 'MACDAVD_117_.npy', 'MACDAVD_118_.npy', 'MACDAVD_119_.npy', 'MACDAVD_120_.npy', 'MACDAVD_121_.npy', 'MACDAVD_122_.npy', 'MACDAVD_123_.npy', 'MACDAVD_124_.npy', 'MACDAVD_125_.npy', 'MACDAVD_126_.npy', 'MACDAVD_127_.npy', 'MACDAVD_128_.npy', 'MACDAVD_129_.npy', 'MACDAVD_130_.npy', 'MACDAVD_131_.npy', 'MACDAVD_132_.npy', 'MACDAVD_133_.npy', 'MACDAVD_135_.npy', 'MACDAVD_136_.npy', 'MACDAVD_137_.npy', 'MACDAVD_138_.npy', 'MACDAVD_112_.npy', 'MACDAVD_134_.

(None, None)

In [7]:
#img = np.load(os.path.join(control_path, os.listdir(control_path)[0]))
#seg = np.load(os.path.join(control_seg_path, os.listdir(control_seg_path)[0]))

#img = np.load("/usr/bmicnas02/data-biwi-01/jeremy_students/previous_work/nicolas/data/freiburg/3/10000000/10000001/image.npy")
#seg = np.load("/usr/bmicnas02/data-biwi-01/jeremy_students/previous_work/nicolas/data/freiburg/3/10000000/10000001/random_walker_prediction.npy")
model_path = '/usr/bmicnas02/data-biwi-01/jeremy_students/lschlyter/CNN-segmentation/logdir/unet3d_da_0.0nchannels4_r1_loss_dice_cut_z_True_full_run_bern_full'
save_path_pred = model_path + "/results/visualization/Bern_patient_level_prediction/"
file_name = 'freiburg_model_prediction_105.npy'
seg = np.load(os.path.join(save_path_pred, file_name))
img = np.load(os.path.join(control_path, os.listdir(control_path)[4]))

In [8]:
img.shape, seg.shape

((160, 110, 48, 25, 4), (144, 112, 42, 48))

In [ ]:
try:
    debug_in = img
    debug_label = seg
    #slice_ = 0
    print(slice_)
    %matplotlib inline
    plt.imshow(debug_in[:, :,slice_, 3,0])
    plt.show()
    plt.imshow(debug_in[:, :,slice_, 3,0])
    plt.imshow(debug_label[:, :,slice_, 3], alpha= 0.1)
    slice_ +=1 
    plt.show()
except:
    pass

##### Masked data 
- I am loading both the Bern data and also some from Freiburg with the original script to make sure I am on track

In [ ]:
basepath = "/usr/bmicnas02/data-biwi-01/jeremy_students/data/inselspital/kady"
#masked_data = h5py.File(basepath + '/masked_images_from_101_to_104.hdf5','r')
masked_data = load_masked_data(basepath, 101, 104, 'train')
images_masked = masked_data['masked_images_train']
labels_masked = masked_data['masked_images_train']

In [ ]:


masked_data_fr = h5py.File("/usr/bmicnas02/data-biwi-01/jeremy_students/previous_work/nicolas/data/freiburg/masked_images_from20to26.hdf5",'r')
images_masked_fr = masked_data_fr['masked_images_validation']

In [ ]:
images_masked.shape, labels_masked.shape

In [ ]:
import matplotlib
masked_array = np.ma.masked_where(images_masked[:, :,:, 3,1] == 0, images_masked[:, :,:, 3,1])

cmap = matplotlib.cm.viridis  # Can be any colormap that you want after the cm
cmap.set_bad(color='black')


In [ ]:
%matplotlib inline
plt.imshow(images_masked[50, :,:, 3,1])
plt.colorbar()
plt.show()

In [ ]:
masked_array.shape

In [ ]:
fig, axs = plt.subplots(5,5, figsize=(10,10))
ax = axs.ravel()
for i in range(25):
    ax[i].imshow(masked_array[(i+6), :,:], cmap=cmap)
    #ax[i].set_title(i)
    ax[i].axis('off')

In [ ]:
%matplotlib inline
print("Freiburg")
plt.imshow(images_masked_fr[21, :,:, 3,1])
plt.show()

##### Cropped data and sliced orthogonally

In [ ]:
cropped_sliced_data = load_cropped_data_sliced(basepath, 101, 104, 'train', force_overwrite=False)
#cropped_sliced_data = h5py.File(basepath + '/sliced_images_from101_to_104.hdf5','r')
images_cropped_sliced = cropped_sliced_data['sliced_images_train']
     

In [ ]:
cropped_sliced_data_fr = h5py.File("/usr/bmicnas02/data-biwi-01/jeremy_students/previous_work/nicolas/data/freiburg/sliced_images_from20to26.hdf5",'r')
images_cropped_sliced_fr = cropped_sliced_data_fr['sliced_images_validation']

#### Now let's load the masked one and show them side by side

In [ ]:
masked_cropped_sliced_data = load_masked_data_sliced(basepath, 101, 104, 'train', force_overwrite=False)
masked_images_cropped_sliced = masked_cropped_sliced_data['sliced_images_train']

In [ ]:
masked_cropped_sliced_data_fr = h5py.File("/usr/bmicnas02/data-biwi-01/jeremy_students/previous_work/nicolas/data/freiburg/masked_sliced_images_from20to26.hdf5",'r')
masked_images_cropped_sliced_fr = masked_cropped_sliced_data_fr['sliced_images_validation']

In [ ]:
masked_images_cropped_sliced.shape, images_cropped_sliced.shape, masked_images_cropped_sliced_fr.shape, images_cropped_sliced_fr.shape

In [ ]:
256/4

In [ ]:
z_slice = 45
z_slices = np.linspace(0, 64, 18, dtype=int)
time_slice = 2
channel_slice = 1
fig, axs = plt.subplots(6,6, figsize=(14,14))
ax = axs.ravel()
z_i = 0 
masked_array = np.ma.masked_where(masked_images_cropped_sliced[:, :,:, time_slice,channel_slice] == 0, masked_images_cropped_sliced[:, :,:, time_slice,channel_slice])

cmap = matplotlib.cm.viridis  # Can be any colormap that you want after the cm
cmap.set_bad(color='black')

for i in range(36):
    if i %2 == 0:

        ax[i].imshow(images_cropped_sliced[z_slices[z_i], :,:, time_slice,channel_slice])
    else:
        ax[i].imshow(masked_array[z_slices[z_i], :,:], cmap=cmap)
        z_i +=1
    #ax[i].set_title(i)
    ax[i].axis('off')
#axs[0].imshow(images_cropped_sliced[z_slice, :,:, time_slice,channel_slice])
#axs[1].imshow(masked_images_cropped_sliced[z_slice, :,:, time_slice,channel_slice])
plt.show()

In [ ]:
print("Freiburg data") 
fig, axs = plt.subplots(1,2)
axs[0].imshow(images_cropped_sliced_fr[z_slice, :,:, time_slice,channel_slice])
axs[1].imshow(masked_images_cropped_sliced_fr[z_slice, :,:, time_slice,channel_slice])
plt.show()


### Let's create the centerlines 

I think these are the following step:
1. show_center_lines - to get the indexes of the coordinates at which the centerline is located
2. Write these indexes in create_center_lines to create them ? 

In [ ]:
show_center_lines(basepath)

In [ ]:
create_center_lines(basepath = basepath)

In [ ]:
images_cropped_sliced_straighten = load_cropped_data_straightened(basepath, 101, 104, 'train', force_overwrite=False)
images_cropped_sliced_straighten = images_cropped_sliced_straighten["straightened_images_train"]

In [ ]:
(images_cropped_sliced_straighten[:] == 0).all()

In [ ]:



plt.imshow(images_cropped_sliced_straighten[120, :,:, 10,0])
plt.show()

In [ ]:
i = 0
subject_sliced = images_cropped_sliced[i*64:(i+1)*64,:,:,:,:]

In [ ]:
#plot_batch_3d(subject_sliced, channel=0, every_x_time_step=5, out_path= "Validation_Subject_Channel0_" + str(i) + ".png")